In [1]:

from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, G2, GT, pair
from charm.toolbox.ABEnc import ABEnc
from charm.toolbox.symcrypto import SymmetricCryptoAbstraction
from charm.core.math.pairing import hashPair as sha2
from charm.core.engine.util import objectToBytes, bytesToObject
from msp import MSP

from datetime import datetime

#Scapy
from scapy.all import Dot11, RadioTap, sendp

#Schnorr
import secrets, hashlib
from ecpy.curves     import Curve,Point
from ecpy.keys       import ECPublicKey, ECPrivateKey
from ecpy.ecschnorr  import ECSchnorr
from charm.core.math.pairing import ZR
from charm.toolbox.policytree import *

#CBOR Encoding
import cbor2

#KDF - Argon2
import argon2, binascii

import math
import sys

#AES ECB mode
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

debug = False
broadcast = False

In [2]:

# policy parser test cases 
parser = PolicyParser()
attrs = ['3','2']
print("Attrs in user set: ", attrs)    
tree1 = parser.parse("(1 or 2) and (2 and 3))")
print("case 1: ", tree1, ", pruned: ", parser.prune(tree1, attrs))        

tree2 = parser.parse("1 or (2 and 3)")
print("case 2: ", tree2, ", pruned: ", parser.prune(tree2, attrs))

tree3 = parser.parse("(1 or 2) and (4 or 3)")
print("case 3: ", tree3, ", pruned: ", parser.prune(tree3, attrs))

Attrs in user set:  ['3', '2']
case 1:  ((1 or 2) and (2 and 3)) , pruned:  [2, 2, 3]
case 2:  (1 or (2 and 3)) , pruned:  [2, 3]
case 3:  ((1 or 2) and (4 or 3)) , pruned:  [2, 3]


In [3]:
attributes = None
rows = None

In [4]:
class FABEO22CPABE(ABEnc):
    def __init__(self, group_obj, verbose=False):
        ABEnc.__init__(self)
        self.name = "FABEO CP-ABE"
        self.group = group_obj
        self.util = MSP(self.group, verbose)

    def setup(self):
        """
        Generates public key and master secret key.
        """

        if debug:
            print('\nSetup algorithm:\n')

        # pick a random element from the two source groups and pair them
        g = self.group.random(G1)
        h = self.group.random(G2)
        
        e_gh = pair(g, h)
        
        alpha = self.group.random(ZR)

        # now compute various parts of the public parameters
        e_gh_alpha = e_gh ** alpha

        # the master secret and public key
        msk = {'alpha': alpha}
        pk = {'g': g, 'h': h, 'e_gh_alpha': e_gh_alpha}

        return pk, msk

    def keygen(self, pk, msk, attr_list):
        """
        Generate a key for a list of attributes.
        """

        if debug:
            print('\nKey generation algorithm:\n')

        r = self.group.random(ZR)
        h_r = pk['h'] ** r

        sk1 = {}
        for attr in attr_list:
            attrHash = self.group.hash(attr, G1)
            sk1[attr] = attrHash ** r
        
        bHash = self.group.hash(str(self.group.order()+1), G1) # ZR+1
#         print(bHash)
        
        sk2 = pk['g'] ** msk['alpha'] * bHash ** r

        return {'attr_list': attr_list, 'h_r': h_r, 'sk1': sk1, 'sk2': sk2}

    def encrypt(self, pk, msg, policy_str):
        """
        Encrypt a message msg under a policy string.
        """

        if debug:
            print('\nEncryption algorithm:\n')

        policy = self.util.createPolicy(policy_str)
#         print(policy.traverse)
        mono_span_prog = self.util.convert_policy_to_msp(policy)
        attributes = mono_span_prog.keys()
        rows = mono_span_prog.values()
        print(attributes)
        print(rows)
# #         a = mono_span_prog
#         print('mono_span_prog',mono_span_prog)
        num_cols = self.util.len_longest_row
#         print('num_cols',num_cols)
        # pick randomness
        s0 = self.group.random(ZR)
#         print('s0',s0)
        s1 = self.group.random(ZR)
#         print('s1',s1)
 # pick random shares
        v = [s0]
        for i in range(num_cols-1):
            rand = self.group.random(ZR)
            v.append(rand)
#         print('pk[h]',pk['h'])    
        g_s0 = pk['h'] ** s0
#         print('gs0:',g_s0)
#         print('gs0 serialized:',self.group.serialize(g_s0))
        h_s1 = pk['h'] ** s1 
#         print('hs1:',h_s1)
#         print('hs1 serialized:',self.group.serialize(h_s1))
       
#         print("msg:",msg)
#         print('group order',str(self.group.order()+1))
        bHash = self.group.hash(str(self.group.order()+1), G1) # ZR+1
#         print('bHash:',bHash)
#         print(inspect.getsource(self.group.serialize))
        ct = {}
        for attr, row in mono_span_prog.items():
#             print('row:',row)
            attr_stripped = self.util.strip_index(attr)
#             print('attr_stripped:',attr_stripped)
            attrHash = self.group.hash(attr_stripped, G1)
#             print('attrHash:',attrHash)
            len_row = len(row)
#             print('len_row:',len_row)
#             print('v[:len_row]',v[:len_row])
            Mivtop = sum(i[0] * i[1] for i in zip(row, v[:len_row]))
#             print('Mivtop:',Mivtop)
            ct[attr] = (self.group.serialize(bHash ** Mivtop * attrHash ** s1))
#             print(ct[attr])
        # compute the e(g, h)^(As) * m term
        Cp = pk['e_gh_alpha'] ** s0
        Cp = Cp * msg

#         return {'policy': str(policy), 'g_s0': (self.group.serialize(g_s0)).decode("utf-8"), 'h_s1': (self.group.serialize(h_s1)).decode("utf-8"), 'ct': ct, 'Cp': self.group.serialize(Cp)}
        return {'g_s0': (self.group.serialize(g_s0)), 'h_s1': (self.group.serialize(h_s1)), 'Cp': self.group.serialize(Cp), 'policy': str(policy).encode('utf-8'), 'ct': ct}

    def decrypt(self, ctxt, key):
        """
        Decrypt ciphertext ctxt with key key.
        """

        if debug:
            print('\nDecryption algorithm:\n')

        ctxt['policy'] = (self.util.createPolicy(ctxt['policy']))
        nodes = self.util.prune(ctxt['policy'], key['attr_list'])
        # print(nodes)
        if not nodes:
            print ("Policy not satisfied.")
            return None

        prod_sk = 1
        prod_ct = 1
        for node in nodes:
            attr = node.getAttributeAndIndex()
            attr_stripped = self.util.strip_index(attr)  # no need, re-use not allowed

            prod_sk *= key['sk1'][attr_stripped]
            prod_ct *= self.group.deserialize((ctxt['ct'][attr]).encode("utf-8"))
        
        e0 = pair(key['sk2'], self.group.deserialize((ctxt['g_s0']).encode("utf-8")))
        e1 = pair(prod_sk, self.group.deserialize((ctxt['h_s1']).encode("utf-8")))
        e2 = pair(prod_ct, key['h_r'])

        kem = e0 * (e1/e2)

        return (self.group.deserialize(ctxt['Cp'])) / kem

    def make_rid(self, drone_id, drone_data, ctxt, timestamp, es):
        """
        # Remote ID message Structure:
        - id: A unique identifier of the identity of the drone.
        - dd: An indication of the drone's current location, expressed in terms of latitude, longitude, geometric altitude, speed and Course Over Ground (COG).
        - gc: The indication of the "encrypted" current location of the control station piloting the drone, expressed in terms of latitude, longitude, and geometric altitude.
        - ts: A timestamp of the message.
        - es: An indicator of the emergency status of the drone.
        """
        
        rid = { "id":           drone_id, 
                "dd":           drone_data, 
                "gc":           ctxt, 
                "ts":           timestamp, 
                "es":           es
            }
        return rid

    def schnorr_setup(self):
        """
        Generates public key and private key for the Schnorr Signatures.
        """
        cv      = Curve.get_curve('secp256k1')
        pv_key  = ECPrivateKey(secrets.randbits(32*8), cv)
        pb_key  = pv_key.get_public_key()
        signer  = ECSchnorr(hashlib.sha256, "ISO", 'ITUPLE')
        return pb_key, pv_key, signer

    
    def schnorr_sign(self, msg, pv_key, signer):
        sig    = signer.sign(str.encode(str(msg)), pv_key)
        if debug: 
            print ('[+] Signature (r): ' + hex(sig[0]))
            print ('[+] Signature (s): ' + hex(sig[1]))
        
        lr = int(math.ceil(len(hex(sig[0])[2:])/2))
        ls = int(math.ceil(len(hex(sig[1])[2:])/2))
        
        r = sig[0].to_bytes(lr, 'big')
        s = sig[1].to_bytes(ls, 'big')
        return (r,s)
    
    
    def schnorr_verify(self, msg, sig, pb_key, signer):
        v = signer.verify(str.encode(str(msg)), (int.from_bytes(sig[0],"big"),int.from_bytes(sig[1],"big")), pb_key)
        if(v == True):
            if debug:
                print("\n[+] Successful Signature Verification")
        else:
            if debug:
                print("\n[x] Error in Signature Verification")
        
        return v

    # create policy string and attribute list for a boolean formula of the form "1 and 2 and 3"
    def create_policy_string_and_attribute_list(self, n):
        policy_string = '(1'
        attr_list = ['1']
        for i in range(2,n+1):
            policy_string += ' and ' + str(i)
            attr1 = str(i)
            attr_list.append(attr1)
        policy_string += ')'

        return policy_string, attr_list


In [5]:
#Helper functions

def aes_ecb_encrypt(key, data):
    cipher = AES.new(key, AES.MODE_ECB)
    ciphertext = cipher.encrypt(data)
    return ciphertext

def aes_ecb_decrypt(key, ciphertext):
    cipher = AES.new(key, AES.MODE_ECB)
    plaintext = cipher.decrypt(ciphertext), AES.block_size
    return plaintext

def minimize_policy_string(input_bstring):
    input_string = input_bstring.decode()
    # Remove brackets and spaces
    modified_string = input_string.replace("(", "").replace(")", "").replace(" ", "")

    # Replace 'and' with '&' and 'or' with '|'
    modified_string = modified_string.replace("and", "&").replace("or", "|")

    return modified_string.encode()



In [6]:

groupObj = PairingGroup('BN254')
print(groupObj.pair_prod)
fabeo22_cp = FABEO22CPABE(groupObj)

# Setup Phase
(pk, mk) = fabeo22_cp.setup()
print("[+] CP-ABE Authority Keys:")
print("pk => ", pk)
print("mk => ", mk)

# Generate Public and Private Key to be used with Schnorr
(pb_sig, pv_sig, signer) = fabeo22_cp.schnorr_setup()

print("\n[+] EC Schnorr Keys:")
print("pk => ", pb_sig)
print("pv => ", pv_sig)



<bound method PairingGroup.pair_prod of <charm.toolbox.pairinggroup.PairingGroup object at 0x7fdcb805a890>>
[+] CP-ABE Authority Keys:
pk =>  {'g': [5245538554028946766663183829148335519083297288368850284342511436381732083095, 6440785069472960250166205232989042836843988332342233333758648152003924003195], 'h': [[11236263642474494046230881573812204000093951929400413974230081189336749754408, 15192751185481641680074089610747488865085983209850111765315840941262072047178], [4217537785231253393425928150921010930277260678535756857768563889857633483011, 5780751142002250659731921720758817130649949345434208329748672665145748238509]], 'e_gh_alpha': [[8446580068701098715330141551190541543155376087418860187541149397202436367200, 15005503995139354493323583854624257254453038870148931687984060144330693892257], [11600955478284618530619608984617739654477657018982106970332317846870238055647, 4580772043856157812656388825078634023405596877697531476176000549365105045863], [92091897136325774178019414543384627

In [7]:
eghA= [[[0x1e00e2c862462008c6fc2b2af9b3c27f662c7f37c7110883e5d30a3f9a9173a3,0x1f828b43f7ac3108481f8646b8f44411c9b50adb1599b595375de6209379215d],[0x099a3a64c41d61f20e1aaf080f6e24c6afeaa58293a6d9fe3112e31bc60985fd,0x096b126bf8651fb85c95bbf02ea5d5d39c6336232e7eeb8180031dacc9fb3e56]],[[0x0df8ef086df2112678c01bbc922ebe34b1c3a77bccb19db7a760162f2b88cd76,0x1e93f4612833e486170422ee5927df174e6816fb65ae54c6077ba231f729d35f],[0x118ae21fd770d73626515569335dfdf59eedb0c1b7d75090908bc4f0d42245e3,0x246134fc951487187141ba17945bb9903a4d7eea15117c83567e25ea55cde9b3]],[[0x0b3b723e7d1571261bbc9309380cdb3ce18d18710adf54dce70c465133689381,0x16ccc01d16960b3fd5c34ee0ab6d4db6f7a07c9c09cb825523d54d2b262ea0ee],[0x15d24e58e70bde816904a103d2dacb87b531f1cd7d54523d5d3dfc690798bcc7,0x0b5a56f1783606b9170f695a5a61b109ab17fd8ae66a2ec0460a425ae12b776c]]]


def int_to_hex(int_value):
    # Convert int to hex and remove the "0x" prefix
    return hex(int_value)[2:]

def hex_to_bytes(hex_string):
    # Pad the hex string to be a multiple of 64 characters (32 bytes)
    hex_string = hex_string.zfill(64)
    # Split the string into 2 character (1 byte) chunks
    bytes = [hex_string[i:i+2] for i in range(0, len(hex_string), 2)]
    return bytes

# Populate your eghA array here

converted_eghA = [[hex_to_bytes(int_to_hex(i)) for i in inner] for inner in eghA]

# Flatten the list of lists
flattened_eghA = [item for sublist in converted_eghA for item in sublist]

# Iterate over each chunk
for chunk in flattened_eghA:
    print("{", end="")
    # Iterate over each byte in the chunk
    for byte in chunk:
        # Print in the specified format
        print(f"0x{byte}, ", end="")
    print("};")


TypeError: 'list' object cannot be interpreted as an integer

In [8]:

access_policy23 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 and A10 and A11 or A12 and Op1 or A41 and A51 and A61 or A71 or A81 and A91 or A101 and A111 or Gov' #23
access_policy22 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 and A10 or A11 and A12 or B1 and B2 or Op1 or Op2 or CAA and TUE and FBI and CIA or Gov' #22
access_policy21 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 and A10 or A11 and A12 or B1 and B2 or Op1 or Op2 or CAA and TUE and FBI or Gov' #21
access_policy20 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 and A10 or A11 and A12 or B1 and B2 or Op1 or Op2 or CAA and TUE or Gov' #20
access_policy19 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 and A10 or A11 and A12 or B1 and B2 or Op1 or Op2 or CAA and Gov' #19
access_policy18 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 and A10 or A11 and A12 or B1 and B2 or Op1 or Op2 and Gov' #18
access_policy17 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 and A10 or A11 and A12 or B1 and B2 or Op1 and Gov' #17
access_policy16 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 and A10 or A11 and A12 or B1 and B2 or Gov' #16
access_policy15 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 and A10 or A11 and A12 or B1 or Gov' #15
access_policy14 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 and A10 or A11 and A12 or Gov' #14
access_policy13 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 and A10 or A11 or Gov' #13
access_policy12 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 and A10 or Gov' #12
access_policy11 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 or Gov' #11
access_policy10 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 or Gov' #10
access_policy9 = 'NL or US and Civ and Op and A4 and A5 and A6 or A7 or Gov' #9
access_policy8 = 'NL or US and Civ and Op and A4 and A5 and A6 or Gov' #8
access_policy7 = 'NL or US and Civ and Op and A4 and A5 or Gov' #7
access_policy6 = 'NL or US and Civ and Op and A4 or Gov' #6
access_policy5 = 'NL or US and Civ and Op or Gov' #5
access_policy4 = 'NL or US and Civ or Gov' #4
access_policy3 = 'NL or US or Gov' #3
access_policy2 = 'NL or GOV' #2
access_policy1 = 'Gov'
access_policies = [
    access_policy1,
    access_policy2,
    access_policy3,
    access_policy4,
    access_policy5,
    access_policy6,
    access_policy7,
    access_policy8,
    access_policy9,
    access_policy10,
    access_policy11,
    access_policy12,
    access_policy13,
    access_policy14,
    access_policy15,
    access_policy16,
    access_policy17,
    access_policy18,
    access_policy19,
    access_policy20,
    access_policy21,
    access_policy22,
    access_policy23
]

In [17]:

# Keygen Phase 

# Simple Attribute List and Access Policy

# attr_list = ['D','E','F','G','T','Y']
attr_list = ["NL","A10","A11","GOV"]
# access_policy = "A and B or C and D"




for access_policy in access_policies:
    # Your code for the current iteration using the access_policy
    # ...
    # Rest of your code

    print("policy:{\"",access_policy,"\"}")
    
#     access_policy = access_policy+i
    sk = fabeo22_cp.keygen(pk, mk, attr_list)
    if debug: 
        print("\n[+] CP-ABE Receiver Keys:")
        print("sk => ", sk)
    t0=parser.parse(access_policy)
#     print(t0)
#     print(parser.prune(t0,attr_list))

    ## Operator Data - 4 bytes for latitude, 4 bytes for longitude, 4 bytes for altitude

    message = b'\xAB\xCD\xEF\xFE\xED\xDE\xAD\xBA\xDD\xAD\xBE\xEF\xDD\xAD\xBE\xEF'
#     print(message.hex())

    import inspect
    # from flask import Flask, send_file, request
    # app = Flask(__name__)
    # @app.route('/myfile.txt', methods=['GET'])
    # def process_get_request():
    k_arr =[]
    ct_arr =[]
    menc_arr=[]

    for i in range(1):
        # Generate a random nonce namely r
        r = groupObj.random(GT)
#         if(groupObj.ismember(r)):
#             print("r is member of group")
#         print("r: ",r)
#         print("r in bytes",len(objectToBytes(r,groupObj))," ",objectToBytes(r,groupObj))
        # ABE Encrypt
        cypher_t = fabeo22_cp.encrypt(pk, r, access_policy)
    #     if debug: 
    #     print("\n[+] Ciphertext:")
    #     print("ct => ", ct)
        print(attributes)
#         print("{")
#         for at in attributes:
#             print("\"" + at , "\",")
        print(rows)
        # Key Derivation Function with Argon2
        kenc = (argon2.hash_password_raw(time_cost=10, memory_cost=2**12, parallelism=2, hash_len=32,password=objectToBytes(r,groupObj), salt=b'public_salt', type=argon2.low_level.Type.ID))
#         print('kenc: ',(kenc))
        k_arr.append(kenc)
    #     ct_arr.append(cbor2.dumps(ct))
    #     print("kenc"+str(i)+" = ",kenc)
    #     # Symmetric Encryption - It encrypts the data with AES in CBC mode with a random IV and PKCS#7 padding
    #     symcrypt    = SymmetricCryptoAbstraction(kenc)

        menc  = aes_ecb_encrypt((kenc),message)
    #     print('menc',menc)
        menc_arr.append(menc)

    #     cbored_data = cbor2.dumps(cypher_t)
    #     print(f"CBOR2 size: {len(cbored_data)} bytes")
        cypher_t_bytes= cypher_t['g_s0']+cypher_t['h_s1']+cypher_t['Cp']+minimize_policy_string(cypher_t['policy'])
        for key  in cypher_t['ct']:
            cypher_t_bytes=cypher_t_bytes+cypher_t['ct'][key]
    #     print(cypher_t_bytes)
    #     print(f"direct size: {len(cypher_t_bytes)} bytes")    
        ct_arr.append(cypher_t_bytes)
    #     print("menc"+str(i)+" = ",menc)




#     with open('data13', 'wb') as f:
#         for kenc,c in zip(k_arr,ct_arr):
#             f.write(kenc+b','+c)
#     #         f.write(kenc)
#             f.write(b'\n')
#     print("Data written to file.")
#     # if __name__ == '__main__':
#     #     app.run(host='0.0.0.0', port=8080)

policy:{" Gov "}
dict_keys(['GOV'])
dict_values([[1]])
None
None
policy:{" NL or GOV "}
dict_keys(['NL', 'GOV'])
dict_values([[1], [1]])
None
None
policy:{" NL or US or Gov "}
dict_keys(['NL', 'US', 'GOV'])
dict_values([[1], [1], [1]])
None
None
policy:{" NL or US and Civ or Gov "}
dict_keys(['NL', 'US', 'CIV', 'GOV'])
dict_values([[1, 1], [1, 1], [0, -1], [1]])
None
None
policy:{" NL or US and Civ and Op or Gov "}
dict_keys(['NL', 'US', 'CIV', 'OP', 'GOV'])
dict_values([[1, 1, 1], [1, 1, 1], [0, 0, -1], [0, -1], [1]])
None
None
policy:{" NL or US and Civ and Op and A4 or Gov "}
dict_keys(['NL', 'US', 'CIV', 'OP', 'A4', 'GOV'])
dict_values([[1, 1, 1, 1], [1, 1, 1, 1], [0, 0, 0, -1], [0, 0, -1], [0, -1], [1]])
None
None
policy:{" NL or US and Civ and Op and A4 and A5 or Gov "}
dict_keys(['NL', 'US', 'CIV', 'OP', 'A4', 'A5', 'GOV'])
dict_values([[1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [0, 0, 0, 0, -1], [0, 0, 0, -1], [0, 0, -1], [0, -1], [1]])
None
None
policy:{" NL or US and Civ and Op and A4

None
None
policy:{" NL or US and Civ and Op and A4 and A5 and A6 or A7 and A8 and A9 and A10 and A11 or A12 and Op1 or A41 and A51 and A61 or A71 or A81 and A91 or A101 and A111 or Gov "}
dict_keys(['NL', 'US', 'CIV', 'OP', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'OP1', 'A41', 'A51', 'A61', 'A71', 'A81', 'A91', 'A101', 'A111', 'GOV'])
dict_values([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, -1], [0, 0, 0, 0, 0, 0, 0, 0, -1], [0, 0, 0, 0, 0, 0, 0, -1], [0, 0, 0, 0, 0, 0, -1], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, -1], [1, 1, 1, 1, 1], [0, 0, 0, 0, -1], [0, 0, 0, -1], [1, 1, 1], [1, 1, 1], [0, 0, -1], [1, 1], [0, -1], [1]])
None
None


In [ ]:
t1 = 6955394408396420376205894265260684436733705049364696196756663513261476610192
t2 = 11402715412525151220580140855613464332015271224619091650598295894755063996572
print(t1+t2)
print(t1+t2-2074847271923970376588027002634262741841312874919744395971218651715234518963)

In [ ]:
for i in range(2,14):
    print("'data"+str(i)+"',")

In [ ]:
import zipfile

def zip_files(file_names, zip_name):
    # Create a new ZIP file
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        # Add each file to the ZIP
        for file in file_names:
            zipf.write(file)

# Usage
files_to_zip = ['data2',
'data3',
'data4',
'data5',
'data6',
'data7',
'data8',
'data9',
'data10',
'data11',
'data12',
'data13']
zip_files(files_to_zip, 'my_archive.zip')

In [ ]:
print(binascii.hexlify(kenc))

In [ ]:
def count_commas_and_newlines(file_path):
    comma_count = 0
    newline_count = 0

    with open(file_path, 'r') as file:
        for line in file:
            comma_count += line.count(',')
            newline_count += line.count('\n')

    return comma_count, newline_count

# Example usage
file_path = 'example.txt'
comma_count, newline_count = count_commas_and_newlines('data.cbor')
print("Number of commas:", comma_count)
print("Number of new lines:", newline_count)



In [ ]:
print(k_arr[0])


In [ ]:
hex_string = "08020000FFFFFFFFFFFF123456789ABCDEADBEEFFEEDDEADFEED0A00010007E507E514000A00010007E507E5140060D06FA3C9275D211BDDFA17285A8765323A435147366F2B376A49596569414869527737495373596D54574D37486C71744B4747762F694E4A6B3264454E4A4533466C6E4A2F49776364564E71553041633836386D43717A4B636B3250584B593561486D56377667413D323A454B6242724441556B796251505233316338796530634970716877784736774950584F4537544E767932416A4C4841496A4B546F4D2F63616862656656796E385678736859357A6F6E764F723249504357357A6C5151453D333A4568774861485355504372715552792B77674C7879526C366A697A63766C664B754A3672332F4A34417A41616B6773466C514C507546726B7635467A356C465139392B69466A465272626469625A5844364B552B446751533746533057796E742B784D79466C3958324A556D6E54533845436333553044522F64396A7A736E61417A6C47597830526E4A6B35503937694E622B38387A504744304931412F6C2F684D7668634D7575775A6B5A5666466848434F4A58426B7A582F30396D454869564E36573339457469526B3961716B7773593147497945642F727144774B56732F4E56366F644F65386A446F51614B5651724670535767345A4F3837335A627744456C4D346368576445476E30666E3848376D6177366F44354B3367455251572F48456C516F3443616E675133616371387649366D4552466E327759397573706E33594158356C68763157326865396E7034646A75787047724261533963346275496777512F7169704E6A6B2B48714F396D4F71353479786E333135774B4D75444F6E6E4663326C634C736C583550324F47794F4A306C6E7075534D7870656A775252484B4C6F555076554E705468326A45544E2F3779423135425830735237754D50797642532F5A46674B3736796747736C5A62676E6552694D357A6D46365154755175356554534F434761752F41426B4A702B614A2F77746E5041666D724E4C7C555326434956264F502641342641352641367C413726413826413926413130264131317C474F56313A453372315643483442556458692F774A5A77726C7179697A445741522F4B4C373451414E71726F4169694941313A414E52643651544F666B36387A624E716E764F63724B7651444B57476958336835456A552F6C6A48324C4141313A42693135324556316F6E7764774D70654C357A6C69456A76557A58626B4C672F67434A73395561307A4B6742313A4334586749417172344C2F566747704675595672747A6433496749632B7A356F6C563258714A2B4537595542313A477A4D4661664C6736735561446C55392F744C6D454C70683141664A4E586D7A3970727278776772474E6B42313A4142496348584C6B51306C6A7A4A554E4151436761506862504F327A767464575A7635532F3968754C6C4542313A44664B3875444839395045544B663764346A614C436559645858754E4C566B4367557A544148325964495941313A464F5A744E496B7033593651714C62446535522B45484B714C7059395762466967735A4F323751345A434D41313A413561346B76337A7A2B4D425A7448476A514E36585079574B56794D37754F67366B686F5875646649435141313A484C65543955734153364435416F3866786A66795776646C556F4A616E503551573654497153454350793441313A47366C72335352495574316C55495838523543417457306172762F366870672F346F44764A4C427465326F42313A48354A78576A4474537A6734522B55574E7571596B2F34796A534B36645A78364A4B4C6C444936754F6F6741313A443171712B516B394B4E486C484353494A4E5668507678694C47477A3442536839356E2F6B47757A4C46414263B0CD007CFD0EDDC83363A5D4593B175ED9BFD89C0F0F0B3565243EF7FEF032234EF82C03FCE8053E2A0E853C0D4750F2AAB7A7625D003A2747DA2611486C236A795E546100000078"
byte_string = bytes.fromhex(hex_string)
print(byte_string)
size = len(byte_string)
print(size)


In [ ]:

# Receiver Side

## Load the received Frame (in this script, we are using the rid_cbor_snd, but in real application you should parse the received packet)
rid_decoded = cbor2.loads(bytes.fromhex(hex_string))

## Verify Message Signature with Schnorr
keys_to_exclude = set(('sig',))
rid_msg_rec = {k:v for k,v in rid_decoded.items() if k not in keys_to_exclude}
msg_ver = fabeo22_cp.schnorr_verify(rid_msg_rec, rid_decoded['sig'], pb_sig, signer)

# ABE and Symmetric Decryption
if (msg_ver == True):
        ctdec = fabeo22_cp.decrypt(rid_decoded['gc'][0], sk)
        print(ctdec)
        kdec = binascii.hexlify(argon2.hash_password_raw(
        time_cost=10, memory_cost=2**12, parallelism=2, hash_len=32,
        password=objectToBytes(ctdec,groupObj), salt=b'public_salt', type=argon2.low_level.Type.ID))
        print(kdec)
        symcrypt_dec = SymmetricCryptoAbstraction(kdec)
        mdec = symcrypt_dec.decrypt(rid_decoded['gc'][1])
        assert mdec == message, "Failed Decryption!!!"

        if debug: 
            print("\n[+] Successful Decryption!")
            print("\n[+] Data Operator:" + mdec.hex())
        else:
            if debug:
                print("You are not Authorized to Decrypt the Data!")

In [ ]:
from flask import Flask, send_file, request

app = Flask(__name__)

@app.route('/myfile.txt', methods=['GET'])
def process_get_request():
    # Execute code to write data to file here
    
    # Send the file as the response
    
    k_arr =[]
    menc_arr =[]


    for i in range(100):

        # Key Derivation Function with Argon2
        kenc = binascii.hexlify(argon2.hash_password_raw(
        time_cost=10, memory_cost=2**12, parallelism=2, hash_len=32,
        password=objectToBytes(r,groupObj), salt=b'public_salt', type=argon2.low_level.Type.ID))
        k_arr.append(kenc)
        print("kenc"+str(i)+" = ",kenc)
        # Symmetric Encryption - It encrypts the data with AES in CBC mode with a random IV and PKCS#7 padding
        symcrypt    = SymmetricCryptoAbstraction(kenc)
        menc        = symcrypt.encrypt(message)
        menc_arr.append(menc)
    #     print("menc"+str(i)+" = ",menc)



    # Open the file in write mode
    with open('data.txt', 'w') as file:
        # Write each value in a new line
        for kenc, menc in zip(k_arr, menc_arr):
            file.write('kenc:'+kenc.decode()+'\n')
            file.write('menc:'+menc+'\n')

    print("Data written to file.")

    return send_file('data.txt')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)




In [ ]:
print(ct_arr[2])

In [ ]:
import msgpack# print(ct['Cp'])

print(f"raw size gs0: {len(cypher_t['g_s0'])} bytes")
print(f"raw size hs1: {len(cypher_t['h_s1'])} bytes")
print(f"raw size Cp: {len(cypher_t['Cp'])} bytes")
print(f"raw size policy: {len(cypher_t['policy'])} bytes")

for key, value in cypher_t['ct'].items():
    size = sys.getsizeof(value.encode())
    print(f"Size of {key}: {size} bytes")
    
test_data = cbor2.dumps(cypher_t)
print(test_data)
print(f"raw size Cypher_t: {len(cypher_t)} bytes")
print(f"cobr size Cypher_T: {len(test_data)} bytes")

In [ ]:

# Remote ID message

## Drone ID
drone_id    = b'\x0a\x0b\x0e\x0d'

## Drone Data - 4 bytes for latitude, 4 bytes for longitude, 4 bytes for altitude, 4 byte for speed, 4 for Course Over Ground'
drone_data  = b'\x02\x6b\x3f\x3e\x01\x6d\x3e\x3a\x03\x01\x01\x01\x05\x04\x04\x04\x06\x08\x08\x08'

## Emergency Code
em_code     = b'\x64'

# Create Remote ID Data Structure
rid_msg = fabeo22_cp.make_rid(drone_id, 
                                drone_data, 
                                [cypher_t, menc], 
                                (int(datetime.timestamp(datetime.now()))).to_bytes(4, 'little'), 
                                em_code)

# Sign the Remote ID message with Schnorr
msg_sig = fabeo22_cp.schnorr_sign(rid_msg, pv_sig, signer)

# Final Message (TX)
rid_msg['sig'] = msg_sig

if debug: 
    print("\n[+] Remote ID Message:")
    print(rid_msg)

## Encode the Remote ID Message by using CBOR
rid_cbor_snd = cbor2.dumps(rid_msg, datetime_as_timestamp=True, value_sharing=False, canonical=False).hex()


# Boradcast the Wi-Fi frame
# Please put the network card in Monitor Mode and execute the script as superuser --> sudo pyhton3 remote_abe.py
payload = bytes.fromhex(rid_cbor_snd)
ssap = "0" * 4
frame = RadioTap() / Dot11(type=2, subtype=0, addr1="ff:ff:ff:ff:ff:ff", addr2="de:ad:be:ef:fe:ed", addr3="ff:ff:ff:ff:ff:ff") / ssap/payload

if broadcast:
    payload = bytes.fromhex(rid_cbor_snd)
    ssap = "0" * 4
    frame = RadioTap() / Dot11(type=2, subtype=0, addr1="ff:ff:ff:ff:ff:ff", addr2="00:11:22:33:44:55", addr3="ff:ff:ff:ff:ff:ff") / ssap/payload
    sendp(frame, iface="wlan0", count=1, inter=1)


In [ ]:
sendp(frame, iface="wlan0", count=1, inter=1)

In [ ]:

# Receiver Side

## Load the received Frame (in this script, we are using the rid_cbor_snd, but in real application you should parse the received packet)
rid_decoded = cbor2.loads(bytes.fromhex(rid_cbor_snd))

## Verify Message Signature with Schnorr
keys_to_exclude = set(('sig',))
rid_msg_rec = {k:v for k,v in rid_decoded.items() if k not in keys_to_exclude}
msg_ver = fabeo22_cp.schnorr_verify(rid_msg_rec, rid_decoded['sig'], pb_sig, signer)

# ABE and Symmetric Decryption
if (msg_ver == True):
        ctdec = fabeo22_cp.decrypt(rid_decoded['gc'][0], sk)
        print(ctdec)
        kdec = binascii.hexlify(argon2.hash_password_raw(
        time_cost=10, memory_cost=2**12, parallelism=2, hash_len=32,
        password=objectToBytes(ctdec,groupObj), salt=b'public_salt', type=argon2.low_level.Type.ID))
        print(kdec)
        symcrypt_dec = SymmetricCryptoAbstraction(kdec)
        mdec = symcrypt_dec.decrypt(rid_decoded['gc'][1])
        assert mdec == message, "Failed Decryption!!!"

        if debug: 
            print("\n[+] Successful Decryption!")
            print("\n[+] Data Operator:" + mdec.hex())
        else:
            if debug:
                print("You are not Authorized to Decrypt the Data!")

In [ ]:
print("rid_cbor_snd:",rid_cbor_snd)
print("rid_decoded:",rid_decoded)

In [ ]:
import json
import gzip

data = {
    'policy': '(COUNTRY1 or COUNTRY2)',
    'g_s0': '2:CWOcsLbOq461vVOOgFspCAfFkzqlYRDPjOVfGczl2ZUgJOjQO3xDuLEJ8lgUtwOe8M71Jf8wL/rbCH+e/PfOfwE=',
    'h_s1': '2:Iaa9YqzyWDPYKYHZVET6Xs2hBxm5bexSK6fei3OiIH4EcLksZGCztgGhhtDwj0d/+8ku9obwxiEAosr4Ej1VfQA=',
    'ct': {
        'COUNTRY1': '1:IivvfUpEDQG+j64yi+njVjmNBAqG5TddRvvZxyAKaekA',
        'COUNTRY2': '1:BKixFCFqFdQGqTTxuRLaquSkLtDFxIiiwP6kXm3L1C8A'
    },
    'Cp': '3:Ao3qZwbsaKq50p6JquwtXJwKhVRK6KYP7PA8VY+IcmERVFYwE9qFeYtl1xVWmTkxdym50dhPCnzvYwJd+t4HFhqS200EeTUoZ6FDuWzyuOdAjXF1nXQVoR6Eu9TE9y0II9OmsMylEOZiIu2VC5Pek9y7Sf5mXkiQpnUm9w1EvEUC1AATFFRxx6lfgwfNUR0wn8YO4+01D4mcxn9dlpliUB2QGvy/HBku5V284s6P6sBE6jln1mlgOQULnhIS0vhlBHcG5gixJk3+qwYsAWNGfYpG3hXMU/WpUpALCCcD3Ukd2lsK3RBt7IyaBWjrT/Jn6mFBUhzB/aGTWF7fJzZHJRnVvKDnQwGnKRtAiQ0JHi/ynyC/29HnVcmx6Mpwt4NGHOFtXsziIMJiFKCwcBtbHHLdziZzrau+L/S/8NqZ5toU6iW0vyA+pFAcIW8alAp1ng2YgovWUj9RcUXdD5SlCwRRriBKt83pZV18O819KnpaggSDTZijcFxAYG+mi06v'
}

# Convert the JSON object to a string
json_str = json.dumps(data)

# Convert the string to bytes
data_bytes = json_str.encode('utf-8')

# Compress the bytes
compressed_bytes = gzip.compress(data_bytes)

print(f'Original size: {len(data_bytes)} bytes')
print(f'Compressed size: {len(compressed_bytes)} bytes')


In [ ]:
import msgpack

# Original JSON-like data
data = {
    'policy': '(COUNTRY1 or COUNTRY2)',
    'g_s0': '2:CWOcsLbOq461vVOOgFspCAfFkzqlYRDPjOVfGczl2ZUgJOjQO3xDuLEJ8lgUtwOe8M71Jf8wL/rbCH+e/PfOfwE=',
    'h_s1': '2:Iaa9YqzyWDPYKYHZVET6Xs2hBxm5bexSK6fei3OiIH4EcLksZGCztgGhhtDwj0d/+8ku9obwxiEAosr4Ej1VfQA=',
    'ct': {
        'COUNTRY1': '1:IivvfUpEDQG+j64yi+njVjmNBAqG5TddRvvZxyAKaekA',
        'COUNTRY2': '1:BKixFCFqFdQGqTTxuRLaquSkLtDFxIiiwP6kXm3L1C8A'
    },
    'Cp': b'3:Ao3qZwbsaKq50p6JquwtXJwKhVRK6KYP7PA8VY+IcmERVFYwE9qFeYtl1xVWmTkxdym50dhPCnzvYwJd+t4HFhqS200EeTUoZ6FDuWzyuOdAjXF1nXQVoR6Eu9TE9y0II9OmsMylEOZiIu2VC5Pek9y7Sf5mXkiQpnUm9w1EvEUC1AATFFRxx6lfgwfNUR0wn8YO4+01D4mcxn9dlpliUB2QGvy/HBku5V284s6P6sBE6jln1mlgOQULnhIS0vhlBHcG5gixJk3+qwYsAWNGfYpG3hXMU/WpUpALCCcD3Ukd2lsK3RBt7IyaBWjrT/Jn6mFBUhzB/aGTWF7fJzZHJRnVvKDnQwGnKRtAiQ0JHi/ynyC/29HnVcmx6Mpwt4NGHOFtXsziIMJiFKCwcBtbHHLdziZzrau+L/S/8NqZ5toU6iW0vyA+pFAc'
}

# Use msgpack to serialize the data
json_str = json.dumps(data)

# Convert the string to bytes
data_bytes = json_str.encode('utf-8')
msgpacked_data = msgpack.packb(data)
print(f'Original size: {len(data_bytes)} bytes')
# Get the size
print(f"MessagePack size: {len(msgpacked_data)} bytes")
